In [1]:
from qiskit_aer import StatevectorSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit
import numpy as np
from qbench.schemes.postselection import benchmark_certification_using_postselection
from qbench.schemes.direct_sum import benchmark_certification_using_direct_sum

In [2]:
### certification experiment for measurement in Hadamard basis using postselection and direct sum ###

def state_prep():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0,1)
    return circuit.to_instruction()
    
def u_dag():
    circuit = QuantumCircuit(1)
    circuit.h(0)
    return circuit.to_instruction()

def v0_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    
    return circuit.to_instruction()

def v1_dag():
    circuit = QuantumCircuit(1)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.x(0)
    return circuit.to_instruction()

def v0_v1_direct_sum_dag():
    circuit = QuantumCircuit(2)
    circuit.p(-np.pi, 0)
    circuit.ry(-2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

In [3]:
service = QiskitRuntimeService()
simulator = service.least_busy(simulator=False,operational=True)

In [4]:
postselection_result = benchmark_certification_using_postselection(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),
    num_shots_per_measurement=10
)

postselection_result

IBMRuntimeError: 'Failed to run program: \'403 Client Error: Forbidden for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"Job create exceeds open plan job usage limits","code":4317,"solution":"Please wait until the beginning of next month to submit more jobs when your quota will reset.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''

In [ ]:
expected = (1/np.sqrt(2) * np.sqrt(0.95) - 1/np.sqrt(2) * np.sqrt(0.05))**2
expected

In [ ]:
direct_sum_result = benchmark_certification_using_direct_sum(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_v1_direct_sum_dag=v0_v1_direct_sum_dag(),
    num_shots_per_measurement=100000
)

direct_sum_result

In [ ]:
p_succ = expected
print(f"Analytical p_succ = {p_succ}")
print(
f"Postselection: p_succ = {postselection_result}, abs_error ={np.abs(p_succ - postselection_result)}"
)
print(f"Direct sum: p_succ = {direct_sum_result}, abs_error ={np.abs(p_succ - direct_sum_result)}")

In [ ]:
from qbench.schemes.postselection import (
assemble_circuits_certification_postselection,
compute_probabilities_certification_postselection,
)
circuits = assemble_circuits_certification_postselection(
target=0,
ancilla=1,
state_preparation=state_prep(),
u_dag=u_dag(),
v0_dag=v0_dag(),
v1_dag=v1_dag(),
)

In [ ]:
from qiskit_aer.noise import NoiseModel, ReadoutError
error = ReadoutError([[0.75, 0.25], [0.8, 0.2]])
noise_model = NoiseModel()
noise_model.add_readout_error(error, [0])
noise_model.add_readout_error(error, [1])

In [ ]:
keys_ordering = ["u_v0", "u_v1"]

all_circuits = [circuits[key] for key in keys_ordering]

counts_noisy = simulator.run(
    all_circuits,
    backend=simulator,
    noise_model=noise_model,
    shots=100000).result().get_counts()

counts_noiseless = simulator.run(
    all_circuits,
    backend=simulator,
    shots=100000).result().get_counts()

In [ ]:
prob_succ_noiseless = compute_probabilities_certification_postselection(
    u_v0_counts=counts_noiseless[0],
    u_v1_counts=counts_noiseless[1],)

prob_succ_noisy = compute_probabilities_certification_postselection(
    u_v0_counts=counts_noisy[0],
    u_v1_counts=counts_noisy[1],)

In [ ]:
print(prob_succ_noiseless)

In [ ]:
print(prob_succ_noisy)